In [47]:
import cv2
import pyautogui
import numpy as np
import time
from PIL import Image
import math

In [48]:

start_image_path = "./img/SequenceMemory/start.png"
button_image_path = "./img/SequenceMemory/button.png"
exit_button_path = "./img/SequenceMemory/exit.png"
grid_image_path = "./img/SequenceMemory/grid.png"

In [49]:
def find_subimage_on_screen(template_path, threshold=0.8):
    screenshot = pyautogui.screenshot()
    screen_np = np.array(screenshot)
    screen_gray = cv2.cvtColor(screen_np, cv2.COLOR_BGR2GRAY)

    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    if template is None:
        raise FileNotFoundError(f"Template image '{template_path}' not found.")

    result = cv2.matchTemplate(screen_gray, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    if max_val >= threshold:
        template_height, template_width = template.shape
        top_left = max_loc
        bottom_right = (top_left[0] + template_width, top_left[1] + template_height)
        return top_left, bottom_right
    return None

In [70]:
def box_to_screenshot( coords ):
    (x1, y1), (x2, y2) = coords
    width = x2 - x1
    height = y2 - y1
    return ( x1, y1, width, height )

def box_mid_point( coords ):
    x = ( coords[0][0]+coords[1][0] )//2
    y = ( coords[0][1]+coords[1][1] )//2
    return x, y

def get_game_matrix( coords ):
    (x1, y1), (x2, y2) = coords
    width = x2 - x1
    height = y2 - y1
    square_length = ( width//3 + height//3 )//2
    curr_box = ( x1+square_length//2, y1+square_length//2 )
    grid_points = []
    for i in range( 3 ):
        temp_grid_points = []
        for j in range( 3 ):
            grid_points.append( (curr_box[0]+i*square_length, curr_box[1]+j*square_length) )
    return grid_points

def rel_distance_from_white(image, x, y):
    pixel_color = image.getpixel((x, y))
    r, g, b = pixel_color
    max_distance = math.sqrt( 3 * 255**2 )
    return math.sqrt( (255-r)**2 + (255-g)**2 + (255-b)**2 ) / max_distance

def closest_to_white(coordinates, threshold=0.9):
    image = pyautogui.screenshot()
    closest_coordinate = None
    min_distance = float('inf')
    
    for i, (x, y) in enumerate( coordinates ):
        rel_distance = rel_distance_from_white( image, x, y )
        if 1 - rel_distance >= threshold:
            if rel_distance  < min_distance:
                closest_coordinate = i
                min_distance = rel_distance

    return closest_coordinate

def extract_memory_sequence(memory):
    result = []
    previous_value = None
    for value in memory:
        if value is not None and value != previous_value:
            result.append(value)
        previous_value = value
    return result

def play_game( screen_coords, button_coords ):
    pyautogui.moveTo( *box_mid_point( button_coords ) )
    pyautogui.click()

    # Game Started
    grid_box = None
    while True:
        grid_box = find_subimage_on_screen( grid_image_path )
        if grid_box: break
    pyautogui.moveTo( *grid_box[0] )
    grid_coords = get_game_matrix( grid_box )

    # Iterate through levels
    level = 1
    while not find_subimage_on_screen( exit_button_path ):
        memory = []
        last_hit = time.time()
        # Get one memory sequence
        while time.time()-last_hit < 1:
            grid_index = closest_to_white( grid_coords, threshold=0.8 )
            if grid_index is not None: last_hit = time.time()
            memory.append( grid_index )
            time.sleep(0.05)
        sequence = extract_memory_sequence( memory )
        print(f"Level {level}: {sequence}")
        assert len( sequence ) == level, "Something went wrong!"
        for index in sequence:
            grid_coord = grid_coords[ index ]
            pyautogui.moveTo( *grid_coord )
            pyautogui.click()
            time.sleep( 0.1 )
        pyautogui.moveTo( *grid_box[0] )
        level += 1

In [71]:
while True:
    screen_coords = find_subimage_on_screen(start_image_path)
    if screen_coords:
        button_coords = find_subimage_on_screen(button_image_path)
        if button_coords:
            play_game( screen_coords, button_coords )
            break
        else:
            print("Start Button not Found, retrying in 1 second...")
    else:
        print("Test screen not found, retrying in 1 second...")
        time.sleep(1)

Test screen not found, retrying in 1 second...
Test screen not found, retrying in 1 second...
Test screen not found, retrying in 1 second...
Level 1: [3]
Level 2: [3, 8]
Level 3: [3, 8, 5]
Level 4: [3, 8, 5, 1]
Level 5: [3, 8, 5, 1, 3]
Level 6: [3, 8, 5, 1, 3, 7]
Level 7: [3, 8, 5, 1, 3, 7, 2]
Level 8: [3, 8, 5, 1, 3, 7, 2, 4]
Level 9: [3, 8, 5, 1, 3, 7, 2, 4, 3]
Level 10: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0]
Level 11: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4]
Level 12: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6]
Level 13: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0]
Level 14: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7]
Level 15: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7, 6]
Level 16: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7, 6, 3]
Level 17: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7, 6, 3, 1]
Level 18: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7, 6, 3, 1, 2]
Level 19: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7, 6, 3, 1, 2, 7]
Level 20: [3, 8, 5, 1, 3, 7, 2, 4, 3, 0, 4, 6, 0, 7, 6, 3, 1, 2, 7, 5]
Level 21: [3, 8, 5

AssertionError: Something went wrong!